## Tutorials
*  https://github.com/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb



In [ ]:
# Transformers installation
! pip install transformers
from transformers import pipeline
! pip install transformers datasets
! pip install sentencepiece
! pip install sacremoses

Exception ignored in: <function _get_module_lock.<locals>.cb at 0x7f2585870820>
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 207, in cb
KeyboardInterrupt: 
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/usr/local/lib/python3.10/di

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
import torch.nn as nn
warnings.filterwarnings('ignore')

In [ ]:
# Load data
df = pd.read_csv("526892.csv", encoding="cp1252",index_col=0)
print(df)

                                              sentence  label      data  regex
0                  Clinical Allergy - Research Article    1.0  Clinical   True
1      Int Arch Allergy Immunol DOI: 10.1159/000526892    0.0       NaN  False
2    Received: May 23, 2022 Accepted: August 17, 20...    0.0       NaN  False
3    Influence of the COVID-19 Pandemic on the Prev...    1.0       flu   True
4    Yuanhong Liu Shengbo Yang Yilan Zeng Caifeng Y...    0.0       NaN  False
..                                                 ...    ...       ...    ...
351                           Int Arch Allergy Immunol    0.0       NaN  False
352                                                 11    0.0       NaN  False
353                               Outbreak of COVID-19    1.0  Outbreak   True
354                             DOI: 10.1159/000526892    0.0       NaN  False
355                                                      0.0       NaN  False

[356 rows x 4 columns]


In [ ]:
sentences = df.iloc[0:10, 0]
labels = df.iloc[0:10, 1]

In [ ]:
labels.value_counts()

1.0    6
0.0    4
Name: label, dtype: int64

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenized = sentences.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
np.array(padded).shape

(10, 344)

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(10, 344)

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
#labels = labels

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
# parameters = {'C': np.linspace(0.0001, 100, 20)}
# grid_search = GridSearchCV(LogisticRegression(), parameters)
# grid_search.fit(train_features, train_labels)

# print('best parameters: ', grid_search.best_params_)
# print('best scrores: ', grid_search.best_score_)

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression()

In [ ]:
lr_clf.score(test_features, test_labels)

0.6666666666666666

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import StratifiedKFold

clf = DummyClassifier()

skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
scores = cross_val_score(clf, train_features, train_labels, cv=skf)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.417 (+/- 0.17)


In [ ]:
!pip install datasets evaluate transformers[sentencepiece]